**1.显示手写体数字图片经PCA压缩后的二维空间分布**

In [18]:
import pandas as pd 
digits_train=pd.read_csv('/Users/scarlett/repository/projects/digits/optdigits.tra',header=None)
digits_test=pd.read_csv('/Users/scarlett/repository/projects/digits/optdigits.tes',header=None)

# 分割训练数据的特征向量和标记，前64维是feature vector，第65维是标记
X_digits=digits_train[np.arange(64)]
y_digits=digits_train[64]

# 导入PCA
from sklearn.decomposition import PCA
# 初始化一个可将高维向量压缩到二维的PCA
estimator=PCA(n_components=2)
X_pca=estimator.fit_transform(X_digits)

# 显示10类图像经PCA压缩后的二维空间分布
from matplotlib import pyplot as plt

def plot_pca_scatter():
    
    colors=['black','blue','purple','yellow','white','red','lime','cyan','orange','gray']
    
    for i in xrange(len(colors)):
        px=X_pca[:,0][y_digits.as_matrix()==i]
        py=X_pca[:,1][y_digits.as_matrix()==i]
        plt.scatter(px,py,c=colors[i])
    
    plt.legend(np.arange(0,10).astype(str))
    plt.xlabel('First Principle Component')
    plt.ylabel('Second Principle Component')
    plt.show()
    plt_pca_scatter()
    

**2.使用原始像素特征和经PCA压缩重建的低维特征，在相同配置的SVM上分别进行图像识别**

In [20]:
X_train=digits_train[np.arange(64)]
y_train=digits_train[64]
X_test=digits_test[np.arange(64)]
y_test=digits_test[64]

# 导入基于线性核的SVM分类器,建模，预测
from sklearn.svm import LinearSVC
svc=LinearSVC()
svc.fit(X_train,y_train)
y_predict=svc.predict(X_test)

# 特征压缩到20维,并转化原训练特征
estimator=PCA(n_components=20)
pca_X_train=estimator.fit_transform(X_train)
pca_X_test=estimator.transform(X_test)

# 对压缩后的20维特征的训练数据进行建模，并对测试集预测
pca_svc=LinearSVC()
pca_svc.fit(pca_X_train,y_train)
pca_y_predict=pca_svc.predict(pca_X_test)

print pca_y_predict,y_predict

[0 1 1 ... 8 9 8] [0 1 2 ... 8 9 8]


**3.性能评估**

In [21]:
from sklearn.metrics import classification_report

print svc.score(X_test,y_test)
print classification_report(y_test,y_predict,target_names=np.arange(10).astype(str))

print pca_svc.score(pca_X_test,y_test)
print classification_report(y_test,pca_y_predict,target_names=np.arange(10).astype(str))

0.9309961046188091
             precision    recall  f1-score   support

          0       0.98      0.98      0.98       178
          1       0.85      0.95      0.90       182
          2       0.99      0.97      0.98       177
          3       0.92      0.95      0.93       183
          4       0.95      0.97      0.96       181
          5       0.90      0.96      0.93       182
          6       0.99      0.98      0.99       181
          7       0.98      0.91      0.94       179
          8       0.96      0.74      0.83       174
          9       0.82      0.91      0.86       180

avg / total       0.93      0.93      0.93      1797

0.9081803005008348
             precision    recall  f1-score   support

          0       0.97      0.96      0.96       178
          1       0.80      0.91      0.85       182
          2       0.96      0.94      0.95       177
          3       0.96      0.91      0.94       183
          4       0.94      0.96      0.95       181
    

由上发现，经过PCA处理后会损失2%左右的预测准确性，但相比原始数据64维度的特征，使用PCA可降低68.75%的维度、

特点分析：

- 降维/压缩是选取数据具有代表性的特征，在保持数据多样性(Variance)的基础上，规避掉大量的特征冗余和噪声；并可节省模型训练时间，提高综合效率
- 但容易损失一些有用的模式信息